**Meal Nutrition Model**

**(a) Data Extraction and Preprocessing**

In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, Concatenate, Flatten
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Step 1: Load Data
cgm_data = pd.read_csv("cgm_train.csv")
label_train = pd.read_csv("label_train.csv")
demographic_data = pd.read_csv("demo_viome_train.csv")
img_data = pd.read_csv("img_train.csv")
# Step 2: Preprocess CGM Data
def preprocess_cgm(cgm_data):
    # Parse CGM Data into sequences per Subject ID and Day
    cgm_data['CGM Data'] = cgm_data['CGM Data'].apply(eval)  # Convert string to list of tuples
    sequences = []
    subjects_days = []

    for _, row in cgm_data.iterrows():
        seq = [reading[1] for reading in row['CGM Data']]
        sequences.append(seq)
        subjects_days.append((row['Subject ID'], row['Day']))

    return np.array(sequences, dtype=object), subjects_days

cgm_sequences, subjects_days = preprocess_cgm(cgm_data)



This function processes continuous glucose monitoring (CGM) data by parsing the 'CGM Data' column,converting string representations of lists of tuples into actual Python lists of tuples. It extracts glucose readings from the tuples and organizes them into sequences of readings.Each sequence is associated with a unique combination of Subject ID and Day for further analysis.

In [ ]:
import matplotlib.pyplot as plt

# Preprocess CGM Data

def safe_eval(data):
    if isinstance(data, str):
        try:
            return eval(data)
        except Exception as e:
            print(f"Error evaluating data: {data}, error: {e}")
            return None  # Handle cases where eval fails
    elif isinstance(data, list):
        return data  # Already a list, return as-is
    else:
        return None  # For NaN, None, or unexpected types
def preprocess_plot_cgm(cgm_data):
    cgm_data['CGM Data'] = cgm_data['CGM Data'].apply(safe_eval)  # Convert string to list of tuples or handle invalid cases
    grouped_data = cgm_data.groupby('Subject ID')
    return grouped_data

import plotly.graph_objects as go

# Interactive time-series plot
def plot_interactive_cgm(grouped_data):
    for subject_id, group in grouped_data:
        fig = go.Figure()

        for _, row in group.iterrows():
            day = row['Day']
            if isinstance(row['CGM Data'], list):  # Ensure CGM Data is valid
                cgm_values = [reading[1] for reading in row['CGM Data'] if len(reading) > 1]
                fig.add_trace(go.Scatter(
                    y=cgm_values,
                    mode='lines',
                    name=f'Day {day}',
                    line=dict(width=2)
                ))

        fig.update_layout(
            title=f"CGM Data for Subject {subject_id}",
            xaxis_title="Time Steps",
            yaxis_title="Glucose Reading",
            legend_title="Days",
            template="plotly_white"
        )
        fig.show()



# Preprocess and plot
grouped_cgm_data = preprocess_plot_cgm(cgm_data)
plot_interactive_cgm(grouped_cgm_data)


This code preprocesses and visualizes CGM data interactively for each Subject ID. The safe_eval function ensures the 'CGM Data' column is safely converted to valid lists of tuples, handling invalid cases gracefully. The data is grouped by 'Subject ID', and an interactive Plotly time-series plot is generated for each subject, where CGM readings are plotted as line graphs for different days.

In [ ]:
img_data.head()

# img_data.info()

,Subject ID,Day,Image Before Breakfast,Image Before Lunch
0,1,2,"[[[140, 122, 108], [135, 118, 104], [118, 104,...","[[[41, 152, 201], [77, 164, 205], [88, 157, 13..."
1,1,3,"[[[67, 58, 47], [59, 52, 41], [51, 45, 35], [4...","[[[40, 59, 77], [35, 56, 72], [20, 36, 47], [9..."
2,1,4,"[[[199, 195, 193], [198, 193, 192], [196, 192,...","[[[53, 44, 38], [51, 43, 36], [54, 47, 39], [4..."
3,1,5,"[[[149, 121, 80], [157, 128, 86], [159, 130, 8...","[[[30, 28, 28], [20, 18, 17], [31, 27, 23], [2..."
4,1,6,"[[[175, 184, 198], [192, 206, 219], [160, 165,...","[[[74, 85, 100], [59, 69, 81], [73, 84, 96], [..."


In [ ]:
# Step 3: Map with Nutritional Data
def map_data_with_labels(cgm_sequences, subjects_days, label_train):
    label_train['Key'] = label_train['Subject ID'].astype(str) + "_" + label_train['Day'].astype(str)
    mapped_data = []
    labels = []

    for seq, (subject_id, day) in zip(cgm_sequences, subjects_days):
        key = f"{subject_id}_{day}"
        if key in label_train['Key'].values:
            mapped_data.append(seq)
            labels.append(label_train[label_train['Key'] == key]['Lunch Calories'].values[0])

    return np.array(mapped_data, dtype=object), np.array(labels)

mapped_sequences, lunch_labels = map_data_with_labels(cgm_sequences, subjects_days, label_train)


Map with Nutritional Data: This function maps CGM sequences to their corresponding lunch calorie labels based on Subject ID and Day. It creates a unique key for each CGM sequence and checks for its match in the label_train dataset. Matched sequences are added to the mapped data, and their associated calorie values are stored as labels. The output is a pair of arrays: one with the mapped CGM sequences and the other with the corresponding lunch calorie labels.

In [ ]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler

def preprocess_demographics(demographic_data):
    # Step 1: Identify categorical and numeric columns
    categorical_cols = ['Race']  # 'Race' is categorical
    numeric_cols = demographic_data.select_dtypes(include=['int64', 'float64']).columns.tolist()
    numeric_cols = [col for col in numeric_cols if col not in categorical_cols + ['Viome']]

    # Step 2: Process the 'Viome' column
    def process_viome(row):
        try:
            values = np.array([float(x) for x in row.split(',')])  # Convert to numeric array
            return np.mean(values)  # Use mean as the representative value
        except:
            return np.nan  # Handle invalid data gracefully

    demographic_data['Viome'] = demographic_data['Viome'].apply(process_viome)

    # Step 3: Handle missing values in numeric columns
    # Only apply fillna(mean) to numeric columns
    for col in numeric_cols + ['Viome']:
        if col in demographic_data.columns:  # Ensure column exists
            demographic_data[col] = pd.to_numeric(demographic_data[col], errors='coerce')
            demographic_data[col] = demographic_data[col].fillna(demographic_data[col].mean())

    # Step 4: Encode 'Race' using OneHotEncoder
    encoder = OneHotEncoder(sparse_output=False)
    encoded_race = encoder.fit_transform(demographic_data[['Race']])

    # Debugging: Print unique values in 'Race'
    print("Unique values in Race:", demographic_data['Race'].unique())

    # Step 5: Remove the original 'Race' column
    demographic_data = demographic_data.drop(columns=['Race'])

    # Step 6: Add encoded 'Race' features
    demographic_data = pd.concat(
        [demographic_data.reset_index(drop=True),
         pd.DataFrame(encoded_race, columns=encoder.get_feature_names_out(['Race']))],
        axis=1
    )

    # Step 7: Validate and Scale Features
    demographic_data = demographic_data.apply(pd.to_numeric, errors='coerce')  # Ensure numeric format
    demographic_data = demographic_data.fillna(0)  # Fill any remaining NaN with 0
    scaler = StandardScaler()
    scaled_demo = scaler.fit_transform(demographic_data)

    return scaled_demo

# Preprocess the demographic data
demographic_data_scaled = preprocess_demographics(demographic_data)


print(demographic_data.isnull().sum())  # Check for remaining NaN values
print(demographic_data.describe())
# Run the preprocessing function
demographic_data_scaled = preprocess_demographics(demographic_data)


Unique values in Race: ['Hispanic/Latino' 'White' 'African American']
Subject ID                  0
Age                         0
Gender                      0
Weight                      0
Height                      0
Race                        0
Diabetes Status             0
A1C                         0
Baseline Fasting Glucose    0
Insulin                     0
Triglycerides               0
Cholesterol                 0
HDL                         0
Non-HDL                     0
LDL                         0
VLDL                        0
CHO/HDL Ratio               0
HOMA-IR                     0
BMI                         0
Viome                       0
dtype: int64
       Subject ID        Age     Gender      Weight     Height  \
count   36.000000  36.000000  36.000000   36.000000  36.000000   
mean    22.777778  50.000000   0.666667  178.794444  64.151389   
std     14.358892  10.996103   0.478091   32.736401   3.308117   
min      1.000000  22.000000   0.000000  116.800000  

In [ ]:
# Step 5: Split Data for Training
train_sequences, test_sequences, train_labels, test_labels = train_test_split(mapped_sequences, lunch_labels, test_size=0.2, random_state=42)



This function preprocesses the demographic dataset by performing several steps:

It identifies categorical ('Race') and numeric columns, excluding 'Viome' initially.
Processes the 'Viome' column by extracting numeric values and replacing it with their mean.
Handles missing values in numeric columns, filling them with column means.
Encodes the categorical 'Race' column using OneHotEncoder and appends the encoded features to the dataset.
Ensures all features are numeric, fills any remaining missing values with 0, and scales the data using StandardScaler. Finally, it returns the preprocessed and scaled demographic data for further analysis.

In [ ]:
def preprocess_img(cgm_data,columnname):
    # Parse CGM Data into sequences per Subject ID and Day
    # cgm_data['CGM Data'] = cgm_data['CGM Data'].apply(eval)  # Convert string to list of tuples
    sequences = []
    subjects_days = []

    for _, row in cgm_data.iterrows():
        seq = [reading for reading in row[columnname]]
        sequences.append(seq)
        subjects_days.append((row['Subject ID'], row['Day']))

    return pd.DataFrame(sequences),subjects_days

imgbrk,subjects_days_img=preprocess_img(img_data,"Image Before Breakfast")
imglunch,subjects_days_img=preprocess_img(img_data,"Image Before Lunch")

In [ ]:
def map_data_with_labels_img(cgm_sequences, subjects_days, label_train):
    label_train['Key'] = label_train['Subject ID'].astype(str) + "_" + label_train['Day'].astype(str)
    mapped_data = []
    labels = []

    for seq, (subject_id, day) in zip(cgm_sequences, subjects_days):
        key = f"{subject_id}_{day}"
        if key in label_train['Key'].values:
            mapped_data.append(seq)
            labels.append(label_train[label_train['Key'] == key]['Lunch Calories'].values[0])

    return pd.DataFrame(mapped_data), np.array(labels)

mapped_imglunch, lunch_labels = map_data_with_labels_img(imglunch, subjects_days_img, label_train)
mapped_imgbrk, lunch_labels = map_data_with_labels_img(imgbrk, subjects_days_img, label_train)


This function processes image-related data by extracting sequences from a specified column ('columnname'). It organizes the sequences of images (e.g., "Image Before Breakfast" or "Image Before Lunch") for each Subject ID and Day into a DataFrame. Additionally, it associates each sequence with its corresponding Subject ID and Day for further analysis.

In [ ]:
from tensorflow.keras.layers import Dropout, Bidirectional

demographic_features_train, demographic_features_test = train_test_split(demographic_data_scaled, test_size=0.2, random_state=42)



In [ ]:
demographic_data.head()

,Subject ID,Age,Gender,Weight,Height,Race,Diabetes Status,A1C,Baseline Fasting Glucose,Insulin,Triglycerides,Cholesterol,HDL,Non-HDL,LDL,VLDL,CHO/HDL Ratio,HOMA-IR,BMI,Viome
0,1,27,0,133.8,65.00,Hispanic/Latino,1,5.4,91.0,2.5,67.0,216.0,74.0,142.0,130.0,13.0,2.9,0.561728,22.263053,NaN
1,2,49,1,169.2,62.00,Hispanic/Latino,1,5.5,93.0,14.8,61.0,181.0,91.0,90.0,78.0,12.0,2.0,3.398519,30.943704,NaN
2,3,59,1,157.0,64.00,Hispanic/Latino,3,6.5,118.0,17.4,154.0,190.0,74.0,116.0,90.0,31.0,2.6,5.069630,26.946045,NaN
3,5,51,1,172.0,62.50,Hispanic/Latino,3,6.6,144.0,12.9,392.0,269.0,38.0,231.0,157.0,78.0,7.1,4.586667,30.954496,NaN
4,6,51,1,197.0,68.75,White,1,5.2,96.0,6.4,75.0,203.0,72.0,131.0,118.0,15.0,2.8,1.517037,29.300575,NaN


In [ ]:
for col in demographic_data.columns:
    if 'Subject ID' in col:
        print(f"Match found: {col}")

Match found: Subject ID


In [ ]:
demographic_data.columns = demographic_data.columns.str.strip()
print(demographic_data.columns)

Index(['Subject ID', 'Age', 'Gender', 'Weight', 'Height', 'Race',
       'Diabetes Status', 'A1C', 'Baseline Fasting Glucose', 'Insulin',
       'Triglycerides', 'Cholesterol', 'HDL', 'Non-HDL', 'LDL', 'VLDL',
       'CHO/HDL Ratio', 'HOMA-IR', 'BMI', 'Viome'],
      dtype='object')


In [ ]:
# Map Demographic Data to Match CGM and Nutritional Data by Subject ID
subject_to_demo = {subject: demo for subject, demo in zip(demographic_data['Subject ID'], demographic_data_scaled)}

# Expand Demographic Data for Each Day
expanded_demographics = np.array([subject_to_demo[subject_id] for subject_id, day in subjects_days])
sequence_length=100
# Prepare Training and Testing Data
train_sequences_padded = np.array([np.pad(seq, (0, sequence_length - len(seq)), 'constant') for seq in train_sequences])
test_sequences_padded = np.array([np.pad(seq, (0, sequence_length - len(seq)), 'constant') for seq in test_sequences])

train_demographics, test_demographics = train_test_split(expanded_demographics, test_size=0.2, random_state=42)



Map demographic data to CGM data by creating a dictionary mapping Subject IDs to their demographic information.Expand demographic data to match each day of CGM data, ensuring alignment with the Subject ID and Day information.Pad CGM sequences to a fixed length (e.g., 100) to standardize input size for model training. Split the expanded demographic data into training and testing sets using an 80-20 split.


**(b) Data Preparation**

In [ ]:


import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
import ast
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error


class ImputationNet(nn.Module):
    def __init__(self, input_channels=3):
        super(ImputationNet, self).__init__()
        self.conv1 = nn.Conv2d(input_channels, 64, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(64, input_channels, kernel_size=3, padding=1)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.conv2(x)
        return x


class CalorieNetWithImputation(nn.Module):
    def __init__(self, input_height, input_width):
        super(CalorieNetWithImputation, self).__init__()
        self.imputer = ImputationNet(input_channels=3)

        # CNN layers for image data
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Dropout(0.1),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Dropout(0.3),
        )

        # Calculate size after CNN feature extraction
        flattened_size = 128 * (input_height // 4) * (input_width // 4)
        self.fc_image = nn.Linear(flattened_size * 2, 256)  # Combined image features

    def forward(self, before_img, after_img):
        before_img = self.imputer(before_img)
        after_img = self.imputer(after_img)

        before_features = self.features(before_img)
        after_features = self.features(after_img)

        before_features = before_features.view(before_features.size(0), -1)
        after_features = after_features.view(after_features.size(0), -1)

        image_features = torch.cat((before_features, after_features), dim=1)
        image_features = self.fc_image(image_features)

        return image_features  # Return features instead of final predictions


class CalorieMatrixDatasetWithSplit(Dataset):
    def __init__(self, data_frame, img_height, img_width, img_channels=3):
        self.data_frame = data_frame
        self.img_height = img_height
        self.img_width = img_width
        self.img_channels = img_channels

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        before_image_str = self.data_frame.iloc[idx]["Image Before Breakfast"]
        before_image_array = np.array(ast.literal_eval(before_image_str), dtype=np.float32)
        if before_image_array.size == 0:
            before_image_array = np.zeros((self.img_height, self.img_width, self.img_channels), dtype=np.float32)
        before_image_array = before_image_array.reshape(self.img_height, self.img_width, self.img_channels)
        before_image_array = np.transpose(before_image_array, (2, 0, 1))  # (C, H, W)

        after_image_str = self.data_frame.iloc[idx]["Image Before Lunch"]
        after_image_array = np.array(ast.literal_eval(after_image_str), dtype=np.float32)
        if after_image_array.size == 0:
            after_image_array = np.zeros((self.img_height, self.img_width, self.img_channels), dtype=np.float32)
        after_image_array = after_image_array.reshape(self.img_height, self.img_width, self.img_channels)
        after_image_array = np.transpose(after_image_array, (2, 0, 1))

        # Load label (calories)
        label = lunch_labels
        # self.data_frame.iloc[idx]["Lunch Calories"]

        return (torch.tensor(before_image_array), torch.tensor(after_image_array)), torch.tensor(label, dtype=torch.float32)

This code defines a dataset class and neural networks for handling imputed image data and calorie prediction:
1. `ImputationNet`: A neural network that imputes missing image data using convolutional layers.
2. `CalorieNetWithImputation`: Combines imputation and CNN-based feature extraction for 'before' and 'after' meal images.
3. `CalorieMatrixDatasetWithSplit`: A PyTorch dataset class that loads image data (before and after meals) from strings, converts them into arrays, handles missing data by replacing it with zero arrays, reshapes and transposes arrays for CNN input, and returns paired images along with corresponding calorie labels.


In [ ]:
img_height, img_width, img_channels = 64, 64, 3
img_data_test=pd.read_csv("img_test.csv")
# img_data=pd.read_csv("/cgm_train.csv")
    # Train-test split
train_df, val_df = train_test_split(img_data, test_size=0.2, random_state=42)
# train_df = img_data.iloc[train_indices].reset_index(drop=True)
# val_df = img_data.iloc[val_indices].reset_index(drop=True)

train_dataset = CalorieMatrixDatasetWithSplit(train_df, img_height, img_width, img_channels)
val_dataset = CalorieMatrixDatasetWithSplit(val_df, img_height, img_width, img_channels)
test_dataset = CalorieMatrixDatasetWithSplit(img_data_test, img_height, img_width, img_channels)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader=DataLoader(test_dataset,batch_size=32,shuffle=False)
# image_features_train = CalorieNetWithImputation(input_height=img_height, input_width=img_width)

This code splits the image data into training and validation datasets using an 80-20 split. It creates instances of the `CalorieMatrixDatasetWithSplit` class for training, validation, and test datasets, which preprocess the images (e.g., resizing to 64x64x3 dimensions). DataLoaders are then created for each dataset to enable efficient data batching and shuffling during model training and evaluation.


**(c) Multimodal model implementation**

In [ ]:
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, Concatenate, Bidirectional, Add, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K

def attention_block(inputs):
    """Attention mechanism to focus on relevant parts of the sequence."""
    attention_scores = Dense(1, activation='tanh')(inputs)  # Compute scores
    attention_weights = Dense(1, activation='softmax')(attention_scores)  # Normalize scores
    context_vector = Lambda(lambda x: K.sum(x[0] * x[1], axis=1))([inputs, attention_weights])  # Weighted sum
    return context_vector

def build_model(sequence_length, demo_dim, image_feature_dim):
    """
    Build the multimodal model based on the provided parameters.
    """
    # CGM Sequence Input
    cgm_input = Input(shape=(sequence_length, 1), name='cgm_input')
    lstm_out1 = Bidirectional(LSTM(32, return_sequences=True))(cgm_input)
    lstm_out1 = Dropout(0.1)(lstm_out1)
    lstm_out2 = Bidirectional(LSTM(32, return_sequences=True))(lstm_out1)
    lstm_out2 = Dropout(0.2)(lstm_out2)
    lstm_out = Add()([lstm_out1, lstm_out2])  # Residual connection between LSTM layers

    # Attention Mechanism
    attention_out = attention_block(lstm_out)

    # Demographic Features Input
    demographic_input = Input(shape=(demo_dim,), name='demographic_input')
    demographic_dense = Dense(32, activation='relu', kernel_regularizer='l2')(demographic_input)
    demographic_dense = Dropout(0.2)(demographic_dense)

    # Image Feature Input
    image_input = Input(shape=(image_feature_dim,), name='image_input')
    image_dense = Dense(64, activation='relu', kernel_regularizer='l2')(image_input)
    image_dense = Dropout(0.2)(image_dense)

    # Combine all features
    combined = Concatenate()([attention_out, demographic_dense, image_dense])
    combined_dense = Dense(64, activation='relu', kernel_regularizer='l2')(combined)
    combined_dense = Dropout(0.2)(combined_dense)
    output = Dense(1, activation='linear', name='output')(combined_dense)

    # Compile Model
    model = Model(inputs=[cgm_input, demographic_input, image_input], outputs=output)
    return model


This code defines a multimodal neural network model that integrates CGM sequences, demographic data, and image features. The CGM sequence is processed through a Bidirectional LSTM with residual connections and an attention mechanism to focus on relevant time-series features. Demographic and image features are processed separately using dense layers with dropout for regularization. The outputs of all three inputs are concatenated, passed through fully connected layers, and used to predict a single output (e.g., calories). The model is compiled to accept three inputs and produce one output, supporting multimodal data learning.

**(d) Model training**

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import mean_squared_error


class RMSRECallback(tf.keras.callbacks.Callback):
    def __init__(self, train_data, train_labels, val_data, val_labels):
        self.train_data = train_data
        self.train_labels = train_labels
        self.val_data = val_data
        self.val_labels = val_labels
        self.train_rmsre = []
        self.val_rmsre = []

    def on_epoch_end(self, epoch, logs=None):
        train_pred = self.model.predict(self.train_data, verbose=0)
        val_pred = self.model.predict(self.val_data, verbose=0)

        train_relative_errors = (self.train_labels - train_pred) / self.train_labels
        val_relative_errors = (self.val_labels - val_pred) / self.val_labels

        train_rmsre = np.sqrt(np.mean(train_relative_errors ** 2))
        val_rmsre = np.sqrt(np.mean(val_relative_errors ** 2))

        self.train_rmsre.append(train_rmsre)
        self.val_rmsre.append(val_rmsre)

def plot_interactive_rmsre(train_rmsre, val_rmsre):
    epochs = list(range(1, len(train_rmsre) + 1))

    fig = go.Figure()

    # Add training RMSRE line
    fig.add_trace(go.Scatter(
        x=epochs,
        y=train_rmsre,
        mode='lines+markers',
        name='Training RMSRE',
        line=dict(width=2, color='blue')
    ))

    # Add validation RMSRE line
    fig.add_trace(go.Scatter(
        x=epochs,
        y=val_rmsre,
        mode='lines+markers',
        name='Validation RMSRE',
        line=dict(width=2, color='orange')
    ))

    # Customize layout
    fig.update_layout(
        title='Curve: RMSRE',
        xaxis_title='Epochs',
        yaxis_title='RMSRE',
        template='plotly_white',
        legend=dict(title="Metrics"),
        hovermode='x unified'
    )

    fig.show()



This code defines a custom Keras callback class RMSRECallback to monitor the Root Mean Squared Relative Error (RMSRE) for both training and validation datasets at the end of each epoch during model training. It calculates the predictions for the train and validation data, computes the relative errors, and stores the RMSRE values for both datasets in separate lists for analysis and visualization. This helps evaluate model performance in terms of relative prediction accuracy.

**Using Pytorch for Image preparation and preprocess**

In [ ]:
model_pytorch = CalorieNetWithImputation(input_height=64, input_width=64)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  # model.to(device)
# model_pytorch.eval()  # Set to evaluation mode
image_features_train = []
model_pytorch.eval()  # Ensure the model is in evaluation mode

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Ensure PyTorch model is in evaluation mode
model_pytorch.eval()
model_pytorch.to(device)

# Extract image features
image_features_train = []
for (before_img, after_img), _ in train_loader:
    before_img, after_img = before_img.to(device), after_img.to(device)
    with torch.no_grad():
        features = model_pytorch(before_img, after_img)
    image_features_train.append(features.cpu().numpy())

image_features_train = np.concatenate(image_features_train, axis=0)


image_features_val = []
for (before_img, after_img), _ in val_loader:
    before_img, after_img = before_img.to(device), after_img.to(device)
    with torch.no_grad():
        features = model_pytorch(before_img, after_img)
    image_features_val.append(features.cpu().numpy())

image_features_val = np.concatenate(image_features_val, axis=0)

image_features_test = []
for (before_img, after_img), _ in test_loader:
    before_img, after_img = before_img.to(device), after_img.to(device)
    with torch.no_grad():
        features = model_pytorch(before_img, after_img)
    image_features_test.append(features.cpu().numpy())


image_features_test = np.concatenate(image_features_test, axis=0)

cgm_test = pd.read_csv("cgm_test.csv")  # Replace with actual path
demographics_test = pd.read_csv("demo_viome_test.csv")  # Replace with actual path
label_test = pd.read_csv("label_test_breakfast_only.csv")  # Replace with actual path

cgm_sequences_test, subjects_days_test = preprocess_cgm(cgm_test)
demographics_test_scaled = preprocess_demographics(demographics_test)
# Prepare CGM test sequences
# sequence_length = 100
test_sequences_padded_new = np.array([np.pad(seq, (0, sequence_length - len(seq)), 'constant') for seq in cgm_sequences_test])

# test_nutritional_features = map_nutritional_features(subjects_days_test, label_test)

# Map demographic data to Subject IDs for the test set
subject_to_demo_test_new = {subject: demo for subject, demo in zip(demographics_test['Subject ID'], demographics_test_scaled)}

# Expand Demographic Data for Each Day in the test set
aligned_demographics_test_new = np.array([
    subject_to_demo_test_new[subject_id] if subject_id in subject_to_demo_test_new else np.zeros_like(demographics_test_scaled[0])
    for subject_id, _ in subjects_days_test
])
# predicted = combined_model.predict([test_sequences_padded, aligned_demographics_test, image_features_test])
# Create an RMSRE callback instance
rmsre_callback = RMSRECallback(
    train_data=[train_sequences_padded, train_demographics, image_features_train],
    train_labels=train_labels,
    val_data=[test_sequences_padded, test_demographics, image_features_val],
    val_labels=test_labels
)


Unique values in Race: ['Hispanic/Latino' 'African American' 'White']


This code prepares features and data for model evaluation using PyTorch and Keras:
1. The PyTorch model (`CalorieNetWithImputation`) is set to evaluation mode and moved to the appropriate device (CPU/GPU).
2. Image features are extracted for training, validation, and test datasets using the PyTorch model. For each batch of "before" and "after" images, features are computed without gradient tracking and stored.
3. Continuous Glucose Monitoring (CGM) test sequences are preprocessed, padded to a fixed sequence length, and prepared for testing.
4. Demographic test data is preprocessed, scaled, and aligned with the Subject IDs for consistency across days in the test set.
5. All prepared data (CGM sequences, demographic data, and image features) is used to create an RMSRE callback, enabling the monitoring of the model's relative error performance during evaluation.


In [ ]:
from sklearn.model_selection import ParameterGrid
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import numpy as np

def tune_and_train(
    model_fn,
    param_grid,
    train_data,
    train_labels,
    val_data,
    val_labels,
    test_data=None,
    test_labels=None,
    verbose=1
):
    best_params = None
    best_model = None
    best_val_loss = float('inf')
    best_history = None

    param_combinations = list(ParameterGrid(param_grid))

    for params in param_combinations:
        print(f"Testing parameters: {params}")
        # Build model with given parameters
        model = build_model(
    sequence_length=sequence_length,
    # nutritional_dim=train_nutritional.shape[1],
    demo_dim=train_demographics.shape[1],
    image_feature_dim=image_features_train.shape[1]  # Dimension of extracted image features
)

        model.compile(
            optimizer=Adam(learning_rate=params['learning_rate'], clipnorm=params['clipnorm']),
            loss='mse',
            metrics=['mae']
        )

        # Define callbacks
        early_stopping = EarlyStopping(
            monitor='val_loss',
            patience=params.get('patience_es', 25),
            restore_best_weights=True,
            verbose=verbose
        )
        lr_scheduler = ReduceLROnPlateau(
            monitor='val_loss',
            factor=params.get('factor_lr', 0.3),
            patience=params.get('patience_lr', 5),
            min_lr=params.get('min_lr', 1e-6),
            verbose=verbose
        )


        # Train the model
        history = model.fit(
            train_data,
            train_labels,
            validation_data=(val_data, val_labels),
            epochs=params.get('epochs', 100),
            batch_size=params.get('batch_size', 32),
            callbacks=[early_stopping, lr_scheduler,rmsre_callback],
            verbose=verbose
        )

        predicted_val_img = model.predict(val_data)

        # Evaluate validation loss
        val_loss = min(history.history['val_loss'])
        print(f"Validation Loss: {val_loss}")
        rmse = np.sqrt(mean_squared_error(val_labels, predicted_val_img))
        relative_errors = (val_labels - predicted_val_img) / val_labels
        rmsre = np.sqrt(np.mean(relative_errors ** 2))
        # Call the function with your data
        plot_interactive_rmsre(rmsre_callback.train_rmsre, rmsre_callback.val_rmsre)
        print(f"RMSE:{rmse}")
        print(f"RMSRE:{rmsre}")
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_params = params
            best_model = model
            best_history = history

    print("\nBest Parameters:", best_params)

    # If test data is provided, make predictions
    predictions = None
    if test_data is not None:
        predictions = best_model.predict(test_data)

        # If test labels are provided, evaluate the test set performance
        if test_labels is not None:
            test_loss, test_mae = best_model.evaluate(test_data, test_labels, verbose=0)
            print(f"Test Loss: {test_loss}, Test MAE: {test_mae}")

    return best_params, best_model, best_history, predictions


This function performs hyperparameter tuning and model training for a given model function (`model_fn`) and parameter grid (`param_grid`).
1. It iterates over all combinations of hyperparameters specified in `param_grid` using `ParameterGrid`.
2. For each parameter combination, it builds the model with the current parameters, compiles it, and defines callbacks for:
- Early stopping based on validation loss.
- Learning rate reduction on plateau for adaptive learning.
- RMSRE monitoring during training.
3. The model is trained on the provided training data and evaluated on the validation set to compute metrics like validation loss, RMSE, and RMSRE.
4. The best-performing model (based on the lowest validation loss) and its parameters are saved.
5. If test data is provided, the best model is used to make predictions and optionally evaluate performance on the test set.
The function returns the best hyperparameters, the best model, training history, and predictions (if test data is provided).


In [ ]:
#Optimal hyper parameters
param_grid = {

    'learning_rate': [0.1],
    'clipnorm': [1.5],
    'batch_size': [32],
    'epochs': [50],
    'patience_es': [15],
    'patience_lr': [5],
    'factor_lr': [0.8],
    'min_lr': [1e-6]
}


'''
Testing parameters: {'batch_size': 32, 'clipnorm': 0.5, 'epochs': 50, 'factor_lr': 0.8, 'learning_rate': 0.1, 'min_lr': 1e-06, 'patience_es': 10, 'patience_lr': 5}

Parameter set used for the sake of producing results for experiemnts, best parameters found used for final kaggle submisision.
param_grid = {

    'learning_rate': [0.1,0.01],
    'clipnorm': [0.5,1.5],
    'batch_size': [32,64],
    'epochs': [50,100],
    'patience_es': [10,15],
    'patience_lr': [5,10],
    'factor_lr': [0.5,0.8],
    'min_lr': [1e-6]
}

'''



"\nTesting parameters: {'batch_size': 32, 'clipnorm': 0.5, 'epochs': 50, 'factor_lr': 0.8, 'learning_rate': 0.1, 'min_lr': 1e-06, 'patience_es': 10, 'patience_lr': 5}\n\nParameter set used for the sake of producing results for experiemnts, best parameters found used for final kaggle submisision.\nparam_grid = {\n\n    'learning_rate': [0.1,0.01],\n    'clipnorm': [0.5,1.5],\n    'batch_size': [32,64],\n    'epochs': [50,100],\n    'patience_es': [10,15],\n    'patience_lr': [5,10],\n    'factor_lr': [0.5,0.8],\n    'min_lr': [1e-6]\n}\n\n"

**Hyperparameter tuning & Cross Validation**

In [ ]:
from sklearn.model_selection import KFold
from tensorflow.keras.callbacks import EarlyStopping

# Function to perform K-Fold Cross-Validation
# cross validation with 15 folds done to produce experiment results
def cross_validate_model(sequence_length, demo_dim,imag_features, k_folds=5):
    kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)
    fold_no = 1
    mae_per_fold = []
    loss_per_fold = []

    for train_index, val_index in kf.split(train_sequences_padded):
        print(f"Training Fold {fold_no}")
        fold_no += 1
        # Split data for the current fold
        X_train_seq, X_val_seq = train_sequences_padded[train_index], train_sequences_padded[val_index]
        # X_train_nutr, X_val_nutr = train_nutritional[train_index], train_nutritional[val_index]
        X_train_demo, X_val_demo = expanded_demographics[train_index], expanded_demographics[val_index]
        X_train_img, X_val_img = image_features_train[train_index], image_features_train[val_index]
        y_train, y_val = train_labels[train_index], train_labels[val_index]

        # Build the model
        best_params, best_model, best_history, predictions_final = tune_and_train(
        model_fn=build_model,
      param_grid=param_grid,
      train_data=[X_train_seq, X_train_demo, X_train_img],
      train_labels=y_train,
      val_data=[X_val_seq, X_val_demo, X_val_img],
      val_labels=y_val,
      test_data=[test_sequences_padded_new, aligned_demographics_test_new, image_features_test],
      verbose=1)

    return predictions_final



In [ ]:
predictions_final = cross_validate_model(
    sequence_length=sequence_length,
    demo_dim=expanded_demographics.shape[1],
    imag_features=image_features_train.shape[1]
    )

final_resultsnew = pd.DataFrame({
    'row_id': range(len(predictions_final.flatten())),
    'label':predictions_final.flatten()

})

final_resultsnew.to_csv('final_predictions.csv', index=False)
print("Final predictions saved to 'final_predictions.csv'.")



Training Fold 1
Testing parameters: {'batch_size': 32, 'clipnorm': 1.5, 'epochs': 50, 'factor_lr': 0.8, 'learning_rate': 0.1, 'min_lr': 1e-06, 'patience_es': 15, 'patience_lr': 5}
Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/ops/nn.py:545: UserWarning:

You are using a softmax over axis -1 of a tensor of shape (None, 100, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?



7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step - loss: 708397.1250 - mae: 733.0829

/usr/local/lib/python3.10/dist-packages/keras/src/ops/nn.py:545: UserWarning:

You are using a softmax over axis -1 of a tensor of shape (32, 100, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?



7/7 ━━━━━━━━━━━━━━━━━━━━ 13s 553ms/step - loss: 708788.0625 - mae: 731.7104 - val_loss: 81616.7109 - val_mae: 253.5707 - learning_rate: 0.1000
Epoch 2/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 278ms/step - loss: 115786.5312 - mae: 252.7101 - val_loss: 90826.5781 - val_mae: 199.1795 - learning_rate: 0.1000
Epoch 3/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 424ms/step - loss: 88009.6641 - mae: 226.8523 - val_loss: 75542.8047 - val_mae: 231.5546 - learning_rate: 0.1000
Epoch 4/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 237ms/step - loss: 104794.3906 - mae: 259.0482 - val_loss: 81990.5312 - val_mae: 201.1696 - learning_rate: 0.1000
Epoch 5/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 244ms/step - loss: 119843.7656 - mae: 277.7405 - val_loss: 110375.8672 - val_mae: 227.2512 - learning_rate: 0.1000
Epoch 6/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 247ms/step - loss: 88197.0938 - mae: 227.8751 - val_loss: 129871.5938 - val_mae: 241.5913 - learning_rate: 0.1000
Epoch 7/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 245ms/step - loss: 134639.9844 - mae: 283.6601 - val_

...

/usr/local/lib/python3.10/dist-packages/keras/src/ops/nn.py:545: UserWarning:

You are using a softmax over axis -1 of a tensor of shape (None, 100, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?



7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step - loss: 522899.2812 - mae: 617.0865

/usr/local/lib/python3.10/dist-packages/keras/src/ops/nn.py:545: UserWarning:

You are using a softmax over axis -1 of a tensor of shape (32, 100, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?



7/7 ━━━━━━━━━━━━━━━━━━━━ 13s 809ms/step - loss: 509647.8125 - mae: 604.9044 - val_loss: 84079.1172 - val_mae: 264.3186 - learning_rate: 0.1000
Epoch 2/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 245ms/step - loss: 110241.0078 - mae: 259.5027 - val_loss: 75242.9688 - val_mae: 243.3237 - learning_rate: 0.1000
Epoch 3/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 243ms/step - loss: 104597.8438 - mae: 267.5691 - val_loss: 83885.4297 - val_mae: 200.8135 - learning_rate: 0.1000
Epoch 4/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 243ms/step - loss: 90918.6484 - mae: 240.3817 - val_loss: 111908.3750 - val_mae: 302.9171 - learning_rate: 0.1000
Epoch 5/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 242ms/step - loss: 141345.6562 - mae: 315.6450 - val_loss: 66258.2812 - val_mae: 215.4932 - learning_rate: 0.1000
Epoch 6/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 244ms/step - loss: 137252.9531 - mae: 305.9230 - val_loss: 129328.4531 - val_mae: 249.7179 - learning_rate: 0.1000
Epoch 7/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 414ms/step - loss: 117241.8281 - mae: 264.6751 - val

RMSE:257.282206701326
RMSRE:0.46295810155221023

Best Parameters: {'batch_size': 32, 'clipnorm': 1.5, 'epochs': 50, 'factor_lr': 0.8, 'learning_rate': 0.1, 'min_lr': 1e-06, 'patience_es': 15, 'patience_lr': 5}
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Final predictions saved to 'final_predictions.csv'.


While hyperparameter tuning we have also observed values around 0.29 and 0.31 for RMSRE which implies optimization worked in favor of our model.